## Phase 2 (Data Understanding): Ausreißeranalyse

* Autorin: Anna (i3-Versicherung)
* Webseite: [Data Science Training - Kapitel 2](https://data-science.training/kapitel-2/)
* Datum: 23.03.2023

Wir führen die Trainings- und Testdaten zusammen und suchen nach (numerischen) Ausreißern der Attribute Fare, Age und SibSp. Dabei hilft uns die Standardnormalverteilung (mit Mittelwert 0 und Standardabweichung 1). Wenn ein Z-skalierter Attributwert größer als die n-fache Standardabweichung ist, gilt er als Ausreißer. Bei negativen Werten benutzen wir "kleiner als" statt "größer als". Mit Hilfe des Absolutbetrags (abs) können wir beide Fälle auch gemeinsam betrachten. Der Parameter n = 4 liefert meistens gute Ergebnisse.

In [4]:
# Pandas Paket (Package) importieren
#  Datenstrukturen und Datenanalyse, I/O
#  https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd
# NumPy Paket (Package) importieren
#  Mehrdimensionale Datenstrukturen (Vektoren, Matrizen, Tensoren, Arrays), Lineare Algebra
#  https://numpy.org/doc/
import numpy as np

In [5]:
# Trainings- und Testdaten als Pandas Data Frame (df) aus CSV-Dateien laden
#  (KNIME: "CSV Reader")
df_train = pd.read_csv('../../data/titanic/original/train.csv')
df_test  = pd.read_csv('../../data/titanic/original/test.csv')

In [6]:
# Daten zusammenführen
#  (KNIME "Concatenate")
df = pd.concat([df_train, df_test], ignore_index=True)

In [7]:
# Datentypen automatisch konvertieren
#  Diese Umwandlung macht später Probleme, wenn die fehlenden Werte zuvor nicht behandelt wurden.
#df = df.convert_dtypes()

In [8]:
# Funktion (Definition): Den kleinsten (positiven) Ausreißer bestimmen
#  (KNIME: "Numeric Outliers")
# 
# In Anlehnung an:
# https://www.kaggle.com/code/rhythmcam/titanic-zscore-outlier-detection
#
def dst_zscore_outlier(df, col, n=4):
    
    # Initialisierung und Berechnung von Mittelwert und Standardabweichung
    out = [] # Leere Liste
    mn  = np.mean(df[col]) # Mittelwert
    sd  = np.std (df[col]) # Standardabweichung
    
    # Schleife über alle Datenzeilen
    for x in df[col]: 
        z = (x - mn) / sd # z-score (so dass gilt: Mittelwert = 0, Standardabweichung = 1)
        if np.abs(z) > n: # z-score gößer als die n-fache Standardabweichung ?
            out.append(x) # Ausreißer zur Liste hinzufügen
    
    # Rückgabe: Minimum der Ausreißerwerte
    if out:
        return np.min(out)
    else:
        return np.inf # Unendlich bei leerer Liste

In [9]:
# Funktion (Definition): Alle (positiven) Ausreißer bestimmen und ausgeben
#
def dst_outliers(df, col, n=4):
    
    # 1. Den kleinsten Ausreißer bestimmen
    min_outlier = dst_zscore_outlier(df, col, n)
    
    # 2. Nach allen Ausreißern filtern
    df1 = df[df[col] >= min_outlier]
    
    # 3. Die Daten absteigend sortieren (d.h. die größten Ausreißer kommen zuerst)
    df1 = df1.sort_values(by=col, ascending=False)
    
    # 4. Die Anzahl der Ausreißer ausgeben
    print(col, ': Anzahl Ausreißer =', df1.shape[0])
    
    # 5. Die Tabelle mit allen Ausreißern anzeigen
    display(df1)

In [10]:
# Fare
dst_outliers(df, 'Fare')

Fare : Anzahl Ausreißer = 20


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1234,1235,NaN,1,"Cardeza, Mrs. James Warburton Martinez (Charlo...",female,58.0,0,1,PC 17755,512.3292,B51 B53 B55,C
258,259,1.0,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C
679,680,1.0,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C
737,738,1.0,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C
27,28,0.0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S
960,961,NaN,1,"Fortune, Mrs. Mark (Mary McDougald)",female,60.0,1,4,19950,263.0000,C23 C25 C27,S
341,342,1.0,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0000,C23 C25 C27,S
438,439,0.0,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0000,C23 C25 C27,S
88,89,1.0,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0000,C23 C25 C27,S
944,945,NaN,1,"Fortune, Miss. Ethel Flora",female,28.0,3,2,19950,263.0000,C23 C25 C27,S


In [11]:
# Age
dst_outliers(df, 'Age', 2.5) # hier 2,5 (statt 4) Standardabweichungen als maximal erlaubte Abweichung

Age : Anzahl Ausreißer = 10


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
630,631,1.0,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,27042,30.0000,A23,S
987,988,NaN,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,19877,78.8500,C46,S
851,852,0.0,3,"Svensson, Mr. Johan",male,74.0,0,0,347060,7.7750,NaN,S
96,97,0.0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,PC 17754,34.6542,A5,C
493,494,0.0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,PC 17609,49.5042,NaN,C
116,117,0.0,3,"Connors, Mr. Patrick",male,70.5,0,0,370369,7.7500,NaN,Q
672,673,0.0,2,"Mitchell, Mr. Henry Michael",male,70.0,0,0,C.A. 24580,10.5000,NaN,S
745,746,0.0,1,"Crosby, Capt. Edward Gifford",male,70.0,1,1,WE/P 5735,71.0000,B22,S
972,973,NaN,1,"Straus, Mr. Isidor",male,67.0,1,0,PC 17483,221.7792,C55 C57,S
33,34,0.0,2,"Wheadon, Mr. Edward H",male,66.0,0,0,C.A. 24579,10.5000,NaN,S


In [12]:
# SibSp
dst_outliers(df, 'SibSp')

SibSp : Anzahl Ausreißer = 15


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
159,160,0.0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.55,NaN,S
180,181,0.0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.55,NaN,S
201,202,0.0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.55,NaN,S
324,325,0.0,3,"Sage, Mr. George John Jr",male,NaN,8,2,CA. 2343,69.55,NaN,S
792,793,0.0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.55,NaN,S
846,847,0.0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,CA. 2343,69.55,NaN,S
863,864,0.0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.55,NaN,S
1079,1080,NaN,3,"Sage, Miss. Ada",female,NaN,8,2,CA. 2343,69.55,NaN,S
1251,1252,NaN,3,"Sage, Master. William Henry",male,14.5,8,2,CA. 2343,69.55,NaN,S
59,60,0.0,3,"Goodwin, Master. William Frederick",male,11.0,5,2,CA 2144,46.90,NaN,S


### Ergebnis der Ausreißeranalyse

Fare
* 20 Passagiere haben einen Ticket-Preis von mehr als 240 Pfund bezahlt.
* Insbesondere Fam. Cardeza sticht heraus (Luxus-Kabinen B51 B53 B55).
* 17 Passagiere haben nicht für die Überfahrt bezahlen müssen.

Age
* 9 Passagiere sind 67 Jahre oder älter.

SibSp
* Die Großfamilien Sage und Goodwin fallen hier auf.

In [14]:
# B51 B53 B55: Was sind das für extrem teure Kabinen?
# Wer hat diese belegt?
df_check = df[df['Cabin'] == 'B51 B53 B55'] 
display(df_check)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
679,680,1.0,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C
872,873,0.0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
1234,1235,NaN,1,"Cardeza, Mrs. James Warburton Martinez (Charlo...",female,58.0,0,1,PC 17755,512.3292,B51 B53 B55,C


### Inkonsistente (fehlerhafte Daten)

Der Passagier Mr. Frans Olof Carlsson (PassengerId = 873) hat die gleiche Kabinennummer (B51 B53 B55) wie die Fam. Cardeza (s.o.). Er ist allein gereist und hat nur 5 Pfund bezahlt.